# With reference list

In [1]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil'

# Scrapping from Wikipedia cities of Brazil that will be used as the 'reference list'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
regx = re.compile(r'.*\([A-Z]{2}\).*')
ref_cities = [x.text for x in soup.find_all('li') if re.findall(regx, x.text)]

In [2]:
import pandas as pd

# Read the data
df = pd.read_csv('example_non-treated.csv', sep = ";").dropna(axis = 1)
df.head()

,Municípios
0,João Pessoa/PB
1,João Pessoa PB
2,Fortaleza - CE
3,Fortaleza - CE
4,João Pessoa - PB


In [3]:
from string_treatment import treat_referenced

df_corrected_with_ref = df.copy()

# Use the 'reference list' previously created to correct the uncorrected list
df_corrected_with_ref['Municípios'] = treat_referenced(df_corrected_with_ref['Municípios'], 
                                                       ref_cities)
df_corrected_with_ref.head()

,Municípios
0,João Pessoa (PB)
1,João Pessoa (PB)
2,Fortaleza (CE)
3,Fortaleza (CE)
4,João Pessoa (PB)


# Without reference list
- Building the reference list within the elements 

In [4]:
import pandas as pd

df = pd.read_csv('example_non-treated.csv', sep = ";").dropna(axis = 1)
df.head()

,Municípios
0,João Pessoa/PB
1,João Pessoa PB
2,Fortaleza - CE
3,Fortaleza - CE
4,João Pessoa - PB


In [5]:
from string_treatment import treat_unreferenced

df_corrected_without_ref = df.copy()

df_corrected_without_ref['Municípios'] = treat_unreferenced(df_corrected_without_ref['Municípios'].values)

# Comparing results

In [7]:
from string_treatment import treat_referenced, treat_unreferenced

comparison = dict()

comparison['Original'] = df['Municípios'].values

# All metrics available in the library
metrics = ['jaro_winkler_similarity', 'damerau_levenshtein_distance', 'hamming_distance']

for metric in metrics: 
    comparison[f'With reference-{metric}'] = treat_referenced(df['Municípios'].values, ref_cities, metric = metric)
    comparison[f'Without reference-{metric}'] = treat_unreferenced(df['Municípios'].values, metric = metric)
    
pd.DataFrame(comparison)

,Original,With reference-jaro_winkler_similarity,Without reference-jaro_winkler_similarity,With reference-damerau_levenshtein_distance,Without reference-damerau_levenshtein_distance,With reference-hamming_distance,Without reference-hamming_distance
0,João Pessoa/PB,João Pessoa (PB),João Pessoa/PB,João Pessoa (PB),João Pessoa/PB,João Pessoa (PB),João Pessoa/PB
1,João Pessoa PB,João Pessoa (PB),João Pessoa/PB,João Pessoa (PB),João Pessoa/PB,João Pessoa (PB),João Pessoa/PB
2,Fortaleza - CE,Fortaleza (CE),Fortaleza - CE,Fortaleza (CE),Fortaleza - CE,Fortaleza (CE),Fortaleza - CE
3,Fortaleza - CE,Fortaleza (CE),Fortaleza - CE,Fortaleza (CE),Fortaleza - CE,Fortaleza (CE),Fortaleza - CE
4,João Pessoa - PB,João Pessoa (PB),João Pessoa/PB,João Pessoa (PB),João Pessoa/PB,João Pessoa (PB),João Pessoa/PB
...,...,...,...,...,...,...,...
1295,"São Caetano do Sul, SP",São Caetano do Sul (SP),São Paulo - SP,São Caetano do Sul (SP),"São Caetano do Sul, SP",São Caetano do Sul (SP),"São Caetano do Sul, SP"
1296,"Valinhos , SP",Valinhos (SP),"Valinhos , SP",Valinhos (SP),"Valinhos , SP",Valinhos (SP),"Valinhos , SP"
1297,Pedreira (SP),Pedreira (SP),Pedreira (SP),Pedreira (SP),Limeira (SP),Pedreira (SP),Pedreira (SP)
1298,"São Paulo, SP",São Paulo (SP),São Paulo - SP,São Paulo (SP),São Paulo - SP,São Paulo (SP),São Paulo - SP
